# Comedians Agentic demo

## Load the packages

In [ ]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.SemanticKernel, 1.55.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.54.0-alpha"
#r "nuget: AutoGen, 0.2.3"
#r "nuget: AutoGen.SourceGenerator, 0.2.3"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using dotenv.net;

### Load the environment variables

In [ ]:
DotEnv.Load();

var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var deployment = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");

### Get an OpenAIClient

In [ ]:
AzureOpenAIClient GetAzureOpenAIClient()
{                
    if (string.IsNullOrEmpty(apiKey) || string.IsNullOrEmpty(endpoint))
    {
        throw new InvalidOperationException("Azure OpenAI configuration is not properly set.");
    }

    var client = new AzureOpenAIClient(
        new Uri(endpoint),
        new System.ClientModel.ApiKeyCredential(apiKey));

    return client;
}

### Get an OpenAI client

In [ ]:
var client = GetAzureOpenAIClient();

### Configure the first comedian

In [ ]:
var comedian1 = new OpenAIChatAgent(
chatClient: client.GetChatClient(deploymentName: deployment),
name: "comedian1",
temperature:1.0f,
systemMessage: "You are a comedian. Your task is to generate 'knock-knock' jokes. Make sure the jokes are appropriate and funny.")
    .RegisterMessageConnector()
    .RegisterMiddleware(
        async (messages, option, agent, _) =>
        {
            var reply = await agent.GenerateReplyAsync(messages, option);

            if (reply.GetContent()?.ToLower().Contains("complete") is true)
            {
                Console.WriteLine("Teacher has completed the conversation.");
                
                return new TextMessage(
                    Role.Assistant,
                    GroupChatExtension.TERMINATE,
                    from: reply.From
                );
            }

            return reply;
        })
    .RegisterPrintMessage();

In [ ]:
var comedian2 = new OpenAIChatAgent(
chatClient: client.GetChatClient(deploymentName: deployment),
name: "comedian2",
temperature:1.0f,
systemMessage: "You are a comedian. Your task is to generate 'chicken that crossed the road' jokes. Make sure the jokes are appropriate and funny.")
    .RegisterMessageConnector()
    .RegisterMiddleware(
        async (messages, option, agent, _) =>
        {
            var reply = await agent.GenerateReplyAsync(messages, option);

            if (reply.GetContent()?.ToLower().Contains("complete") is true)
            {
                Console.WriteLine("Teacher has completed the conversation.");
                
                return new TextMessage(
                    Role.Assistant,
                    GroupChatExtension.TERMINATE,
                    from: reply.From
                );
            }

            return reply;
        })
    .RegisterPrintMessage();

In [ ]:
var conversation = await comedian1.InitiateChatAsync(
            receiver: comedian2,
            message: @"Tell me a joke",
            maxRound: 10
        );

conversation